## Linking Latin Philosophical Expressions project

In [1]:
from rdflib import Graph, Literal, RDF, URIRef, Namespace
import pandas as pd
import requests
import json
import pyconll
import re

In [2]:
def generate_expression_id(expression):
    expression = expression.lower().replace(",", "").replace("?", "")
    w = expression.split(" ")
    return "_".join(w[ : 4])

### Latin philosophical expression

In [3]:
df_latin = pd.read_csv('./Latin-Philosophical-Expressions.csv', encoding = 'utf-8')

In [4]:
df_latin = df_latin.fillna('')

In [5]:
df_latin.head()

,id,latin_expression,expression_url,branch,branch_url,concept,translation_eng
0,a_dicto_secundum_quid,a dicto secundum quid ad dictum simpliciter,https://www.wikidata.org/wiki/Q2456273,logic,https://www.wikidata.org/wiki/Q8078,The phrase 'a dicto secundum quid ad dictum si...,from a statement qualified to a statement unqu...
1,a_dicto_simpliciter_ad,a dicto simpliciter ad dictum secundum quid,https://www.wikidata.org/wiki/Q4660909,logic,https://www.wikidata.org/wiki/Q8078,The Latin phrase 'a dicto simpliciter ad dictu...,from the saying absolutely to the saying with ...
2,a_fortiori,a fortiori,https://www.wikidata.org/wiki/Q1753631,logic,https://www.wikidata.org/wiki/Q8078,The Latin phrase 'a fortiori' translates to 'f...,a fortiori
3,a_necesse_ad_esse,a necesse ad esse valet consequentia,,logic,https://www.wikidata.org/wiki/Q8078,The Latin phrase 'a necesse ad esse valet cons...,"From necessity to being, the consequence holds."
4,a_posteriori,a posteriori,https://www.wikidata.org/wiki/Q300637,epistemology,https://www.wikidata.org/wiki/Q9471,The term 'a posteriori' is a Latin phrase mean...,from the latter


In [6]:
# Test on small dataset
#expressions_filtered = []
#with open('./expressions_filtered.txt') as f:
#    expressions_filtered = f.read().split("\n")
#df_latin = df_latin[df_latin['latin_expression'].isin(expressions_filtered)]

### Different interpretations (senses)

In [7]:
df_senses = pd.read_csv('./Latin-Expressions-Interpretations.csv', encoding = 'utf-8')

In [8]:
df_senses = df_senses.fillna('')

In [9]:
df_senses.head()

,expression,associated_with,sense,id,associated_with_url
0,a dicto secundum quid ad dictum simpliciter,Aristotle,Aristotle uses this expression to differentiat...,a_dicto_secundum_quid,https://www.wikidata.org/wiki/Q84473023
1,a dicto secundum quid ad dictum simpliciter,Thomas Aquinas,Aquinas interprets this phrase in the context ...,a_dicto_secundum_quid,https://www.wikidata.org/wiki/Q9438
2,a dicto secundum quid ad dictum simpliciter,Immanuel Kant,Kant might interpret this expression in relati...,a_dicto_secundum_quid,https://www.wikidata.org/wiki/Q9312
3,a dicto secundum quid ad dictum simpliciter,Leibniz,Leibniz could view this distinction as a refle...,a_dicto_secundum_quid,https://www.wikidata.org/wiki/Q9047
4,a dicto simpliciter ad dictum secundum quid,Aristotle,Aristotle uses this expression to differentiat...,a_dicto_simpliciter_ad,https://www.wikidata.org/wiki/Q84473023


## Linking to LiLa lemmas

In [10]:
with open("./lila_linking_final.json", "r") as f:
    text = f.read()
lila_linking = json.loads(text)

## Linked concepts

In [11]:
linked_concepts = [
    ["a dicto secundum quid ad dictum simpliciter","a dicto simpliciter ad dictum secundum quid"],
    ["a priori","a posteriori","a fortiori"],
    ["argumentum ad baculum","argumentum ad hominem","argumentum ad populum"],
    ["bellum omnium contra omnes","homo homini lupus"],
    ["modus ponens","modus tollens"],
    ["de dicto","de facto","de jure","de re"],
    ["in actu","in esse","in potentia"],
    ["mundus intelligibilis","mundus sensibilis"],
    ["per accidens","per se"],
    ["res cogitans", "res extensa"],
    ["tabula rasa","nihil in intellectu nisi prius in sensu"]
]

In [12]:
linked_concepts_list = [generate_expression_id(concept)  for concepts in linked_concepts for concept in concepts]

In [13]:
linked_concepts_dict = {c : {'uri':'', 'links' : []} for c in set(linked_concepts_list)}

In [14]:
for concepts in linked_concepts:
    for concept in concepts:
        concept_id = generate_expression_id(concept)       
        for linked_concept in concepts:
            if linked_concept != concept:
                 linked_concepts_dict[concept_id]['links'].append(generate_expression_id(linked_concept))


In [15]:
linked_concepts_dict

{'in_potentia': {'uri': '', 'links': ['in_actu', 'in_esse']},
 'per_se': {'uri': '', 'links': ['per_accidens']},
 'nihil_in_intellectu_nisi': {'uri': '', 'links': ['tabula_rasa']},
 'de_facto': {'uri': '', 'links': ['de_dicto', 'de_jure', 'de_re']},
 'tabula_rasa': {'uri': '', 'links': ['nihil_in_intellectu_nisi']},
 'modus_ponens': {'uri': '', 'links': ['modus_tollens']},
 'de_jure': {'uri': '', 'links': ['de_dicto', 'de_facto', 'de_re']},
 'de_dicto': {'uri': '', 'links': ['de_facto', 'de_jure', 'de_re']},
 'argumentum_ad_baculum': {'uri': '',
  'links': ['argumentum_ad_hominem', 'argumentum_ad_populum']},
 'a_fortiori': {'uri': '', 'links': ['a_priori', 'a_posteriori']},
 'homo_homini_lupus': {'uri': '', 'links': ['bellum_omnium_contra_omnes']},
 'argumentum_ad_populum': {'uri': '',
  'links': ['argumentum_ad_baculum', 'argumentum_ad_hominem']},
 'a_posteriori': {'uri': '', 'links': ['a_priori', 'a_fortiori']},
 'bellum_omnium_contra_omnes': {'uri': '', 'links': ['homo_homini_lupus'

## UDPipe

In [112]:
def call_udpipe_and_parse(text, model='latin-evalatin24-240520'):
    url = 'https://lindat.mff.cuni.cz/services/udpipe/api/process'
    params = {
        'tokenizer': '',
        'tagger': '',
        'parser': '',
        'data': text,
        'model': model
    }

    response = requests.post(url, data=params)
    
    if response.status_code == 200:
        result = response.json()
        conllu_result = result.get('result')
        if conllu_result:
            # Parse CoNLL-U with pyconll
            conll_data = pyconll.load_from_string(conllu_result)
            return conll_data
        else:
            print("No CoNLL-U output found. Error:", result.get('error', 'Unknown error'))
            return None
    else:
        print(f"HTTP Error {response.status_code}: {response.text}")
        return None

connl = call_udpipe_and_parse("homo homini lupus")

HTTP Error 502: <html>
<head><title>502 Bad Gateway</title></head>
<body>
<center><h1>502 Bad Gateway</h1></center>
<hr><center>nginx/1.20.1</center>
</body>
</html>



In [110]:
# Alternative function when UDPipe doesn't respond

In [114]:
import stanza

nlp = stanza.Pipeline(lang='la', processors='tokenize,pos,lemma,depparse')

def call_udpipe_and_parse(text, model=''):
    doc = nlp(text)
    conllu_lines = []
    for i, sentence in enumerate(doc.sentences, 1):
        conllu_lines.append(f"# sent_id = {i}")
        conllu_lines.append(f"# text = {sentence.text}")
        for word in sentence.words:
            line = [
                str(word.id),
                word.text,
                word.lemma or "_",
                word.upos or "_",
                word.xpos or "_",
                "_" if not word.feats else word.feats,
                str(word.head),
                word.deprel or "_",
                "_",
                "_"
            ]
            conllu_lines.append("\t".join(line))
        conllu_lines.append("")  # sentence separator
    
    conllu_result = "\n".join(conllu_lines)
    conll_data = pyconll.load_from_string(conllu_result)
    return conll_data

connl = call_udpipe_and_parse("homo homini lupus")

2025-05-26 17:56:33 INFO: Checking for updates to resources.json in case models have been updated.  Note: this behavior can be turned off with download_method=None or download_method=DownloadMethod.REUSE_RESOURCES


2025-05-26 17:56:33 INFO: Downloaded file to C:\Users\giovannt\stanza_resources\resources.json
2025-05-26 17:56:34 INFO: Loading these models for language: la (Latin):
| Processor | Package       |
-----------------------------
| tokenize  | ittb          |
| pos       | ittb_nocharlm |
| lemma     | ittb_nocharlm |
| depparse  | ittb_nocharlm |

2025-05-26 17:56:34 INFO: Using device: cpu
2025-05-26 17:56:34 INFO: Loading: tokenize
2025-05-26 17:56:34 INFO: Loading: pos
2025-05-26 17:56:34 INFO: Loading: lemma
2025-05-26 17:56:34 INFO: Loading: depparse
2025-05-26 17:56:35 INFO: Done loading processors!


## Create triples

In [25]:
namespace_sep = '/'

In [26]:
def create_uri(namespace, resource_type, identifier, namespace_sep, language=None):
    
    invalid_chars_pattern = r"[^\w]"  # allow alphanumerics    
    identifier = re.sub(invalid_chars_pattern, "-", str(identifier).lower())  
    
    if language:
        # Language-specific resources
        return namespace[f"{resource_type}{namespace_sep}{language}{namespace_sep}{identifier}"]
    else:
        # Language-neutral resources
        return namespace[f"{resource_type}{namespace_sep}{identifier}"]


In [27]:
from rdflib import Graph, URIRef, Literal, Namespace
from rdflib.namespace import RDF, RDFS, OWL, DC, FOAF, SKOS, DCTERMS

# Define namespaces
ONTOLEX = Namespace("http://www.w3.org/ns/lemon/ontolex#")
LEXINFO = Namespace("http://www.lexinfo.net/ontology/2.0/lexinfo#")
LIME = Namespace("http://www.w3.org/ns/lemon/lime#")
VARTRANS = Namespace("http://www.w3.org/ns/lemon/vartrans#")
PROV = Namespace("http://www.w3.org/ns/prov#")
WD = Namespace("http://www.wikidata.org/entity/")
DCT = Namespace("http://purl.org/dc/terms/")
OA = Namespace("http://www.w3.org/ns/oa#")

POWLA = Namespace("http://purl.org/powla/powla.owl#")
LILA = Namespace("http://lila-erc.eu/ontologies/lila/")

# My namespace
LP = Namespace("http://latinphilosophy.org/")

In [108]:
g = Graph()
    
# Binding
g.bind("ontolex", ONTOLEX)
g.bind("lexinfo", LEXINFO)
g.bind("lime", LIME)
g.bind("skos", SKOS)
g.bind("dcterms", DCTERMS)
g.bind("vartrans", VARTRANS)
g.bind("prov", PROV)
g.bind("dct", DCT)
g.bind("powla", POWLA)
g.bind("foaf", FOAF)
g.bind("lila", LILA)
g.bind("oa", OA)

g.bind("lp", LP)

# Corpus
corpus_uri =LP["corpus"]
g.add((corpus_uri, RDF.type, POWLA.Corpus))
g.add((corpus_uri, RDFS.label, Literal('Philosophical Latin Expressions Corpus')))  
g.add((corpus_uri, DCT.creator, Literal('Giovanna Tonazzo')))  

# Document and expressions layers
doc_uri = create_uri(LP,"layer", "document", namespace_sep)
g.add((doc_uri, RDF.type, POWLA.DocumentLayer))
g.add((doc_uri, RDFS.label, Literal('Document Layer')))
g.add((doc_uri, POWLA.hasDocument, corpus_uri))

cit_uri = create_uri(LP, "layer", "expression", namespace_sep)
g.add((cit_uri, RDF.type, POWLA.DocumentLayer))
g.add((cit_uri, RDFS.label, Literal('Expression Layer')))
g.add((cit_uri, POWLA.hasDocument, corpus_uri))

# Annotation layers
ud_layer = create_uri(LP, "layer", "ud", namespace_sep)
g.add((ud_layer, RDF.type, POWLA.Layer))
g.add((ud_layer, RDFS.label, Literal('UD Annotation Layer')))  
                                             
sem_layer = create_uri(LP, "layer", "semantics", namespace_sep)
g.add((sem_layer, RDF.type, POWLA.Layer))
g.add((sem_layer, RDFS.label, Literal('Semantic Annotation Layer')))  
         
dep_layer = create_uri(LP, "layer", "dependency", namespace_sep)
g.add((dep_layer, RDF.type, POWLA.Layer))
g.add((dep_layer, RDFS.label, Literal('Dependency Annotation Layer')))

g.add((corpus_uri, POWLA.hasLayer, ud_layer))
g.add((corpus_uri, POWLA.hasLayer, sem_layer))
g.add((corpus_uri, POWLA.hasLayer, dep_layer))

# Lexicon for Latin and English
lat_lex_uri = create_uri(LP, "lexicon", "la", namespace_sep)
g.add((lat_lex_uri, RDF.type, LIME.lexicon))
g.add((lat_lex_uri, RDFS.label, Literal('Latin lexicon')))
g.add((lat_lex_uri, DCT.language, Literal('la')))

eng_lex_uri = create_uri(LP, "lexicon", "en", namespace_sep)
g.add((eng_lex_uri, RDF.type, LIME.lexicon))
g.add((eng_lex_uri, RDFS.label, Literal('English lexicon')))
g.add((eng_lex_uri, DCT.language, Literal('En')))

# Create  triples for each expression
for index, row in df_latin.iterrows():
    
    expr_id = row['id']
    latin_expr = row['latin_expression']    
    branch = row['branch']
    branch_url = row['branch_url']
    expr_url = row['expression_url']        
    concept = row['concept']
    expr_trans_eng = row['translation_eng']

    print(latin_expr)
    
    # Lexical concept
    concept_uri = create_uri(LP, "concept", index, namespace_sep)
    g.add((concept_uri, RDF.type, ONTOLEX.LexicalConcept))
    g.add((concept_uri, RDFS.label,  Literal(f'Lexical concept of {latin_expr}', lang="en")))
    g.add((concept_uri, SKOS.definition, Literal(concept, lang="en"))) 
    sem_annot_uri = create_uri(LP, 'semantics', index, namespace_sep)
    g.add((sem_annot_uri, RDFS.label, Literal(f'Semantic annotation of {latin_expr}')))
    g.add((sem_annot_uri, RDF.type, POWLA.Node))
    g.add((sem_annot_uri, POWLA.hasLayer, sem_layer))
    g.add((sem_annot_uri, DC.subject, concept_uri))

    if linked_concepts_dict.get(expr_id):
        linked_concepts_dict[expr_id]['uri'] = concept_uri
    
    # Link to branch of philosophy
    branch_id = branch.lower().replace(" ", "-")
    branch_uri = create_uri(LP, "branch", branch_id, namespace_sep)
    g.add((branch_uri, RDF.type, SKOS.Concept))
    g.add((branch_uri, SKOS.prefLabel, Literal(branch.capitalize(), lang="en")))
    g.add((concept_uri, SKOS.broader, branch_uri))   
    if branch_url:   
        g.add((branch_uri, RDFS.seeAlso, URIRef(branch_url)))
        
    # Link to Wikidata if available
    wiki_id = expr_url.split('/')[-1] if expr_url else None    
    if wiki_id:
        g.add((concept_uri, RDFS.seeAlso, WD[wiki_id]))
        
    # Lexical entry for Latin expression
    latin_entry_uri = create_uri(LP, "expression", index, namespace_sep, "la")
    g.add((latin_entry_uri, RDF.type, ONTOLEX.LexicalEntry))
    g.add((latin_entry_uri, RDFS.label, Literal(latin_expr, lang="la")))    
    g.add((latin_entry_uri, RDF.type, POWLA.root))
    g.add((latin_entry_uri, ONTOLEX.evokes, concept_uri))
    
    # Lexical form
    form_uri = create_uri(LP, "form", index, namespace_sep, "la")
    g.add((form_uri, RDF.type, ONTOLEX.Form))
    g.add((latin_entry_uri, ONTOLEX.canonicalForm, form_uri))
    g.add((form_uri, ONTOLEX.writtenRep, Literal(latin_expr.capitalize(), lang="la")))    
   
    # Lexical entry for English translation
    english_entry_uri = create_uri(LP, "expression", index, namespace_sep, "en")
    g.add((english_entry_uri, RDF.type, ONTOLEX.LexicalEntry))
    g.add((english_entry_uri, RDFS.label, Literal(expr_trans_eng, lang="en")))
    g.add((english_entry_uri, ONTOLEX.evokes, concept_uri))
    
    # Canonical form for English
    english_form_uri = create_uri(LP, "form", index, namespace_sep, "en")
    g.add((english_form_uri, RDF.type, ONTOLEX.Form))
    g.add((english_entry_uri, ONTOLEX.canonicalForm, english_form_uri))
    g.add((english_form_uri, ONTOLEX.writtenRep, Literal(expr_trans_eng.capitalize(), lang="en")))
    
    # Translation relation
    trans_uri = create_uri(LP, "translation", index, namespace_sep)
    g.add((trans_uri, RDF.type, VARTRANS.LexicalRel))
    g.add((trans_uri, VARTRANS.category, Literal('philosophicalTranslation')))
    g.add((trans_uri, RDFS.label, Literal(f'Translation of {latin_expr}')))
    
    g.add((trans_uri, VARTRANS.source, latin_entry_uri))
    g.add((trans_uri, VARTRANS.target, english_entry_uri))

    expr_index = f'expr-{index}'
    
    # Different interpretations
    for sense_id , sense in df_senses[df_senses['id'] == expr_id].iterrows(): 
        
        # Lexical sense
        auth_id = sense['associated_with'].lower().replace(" ", "-")
        sense_uri = create_uri(LP, f'sense_{expr_index}', auth_id, namespace_sep)
        g.add((sense_uri, RDF.type, ONTOLEX.LexicalSense))
        g.add((latin_entry_uri, ONTOLEX.sense, sense_uri))   
        g.add((sense_uri, RDFS.label, Literal(f'{sense['associated_with']} sense of {latin_expr}', lang="en")))
        g.add((sense_uri, SKOS.definition, Literal(sense['sense'], lang="en")))
        g.add((concept_uri, SKOS.narrower, sense_uri))

        auth_uri = create_uri(LP, "creator", auth_id, namespace_sep)
        g.add((auth_uri,  RDFS.label, Literal(sense['associated_with'], lang="en")))
        g.add((auth_uri, FOAF.page, URIRef(sense['associated_with_url'])))
        g.add((sense_uri, DCT.creator, auth_uri))   
        g.add((concept_uri, DCT.creator, auth_uri))
        if sense['associated_with_url'] != '':   
            g.add((auth_uri, RDFS.seeAlso, URIRef(sense['associated_with_url'])))       
        
        if auth_id == 'plato':
            plato_uri = auth_uri
            
    # Add to citation layer
    g.add((latin_entry_uri, POWLA.hasLayer, cit_uri))

    # Add to lexicon layer
    g.add((latin_entry_uri, LIME.entry, lat_lex_uri))
    g.add((english_entry_uri, LIME.entry, eng_lex_uri))
    
    # Parse with UDPipe
    conll = call_udpipe_and_parse(latin_expr.replace(",","").replace("?",""))    
    if conll:                              
        for i, token in enumerate(conll[0]):
                
            # Terminal for the token
            token_id = i + 1            
            powla_token_uri = create_uri(LP, f'token{namespace_sep}{expr_index}', token_id, namespace_sep)
            token_form = token.form
                                                    
            g.add((powla_token_uri, RDF.type, POWLA.Terminal))
            g.add((powla_token_uri, RDFS.label, Literal(token_form, lang="la")))
            g.add((latin_entry_uri, POWLA.hasTerminal, powla_token_uri))
            g.add((powla_token_uri, POWLA.hasLayer, doc_uri))
    
            # First, next, last token relations
            if token_id == 1:
                g.add((latin_entry_uri, POWLA.first, powla_token_uri))        
            
            if token_id == len(conll[0]):
                g.add((latin_entry_uri, POWLA.last, powla_token_uri))
    
            if token_id > 1:
                prev_token_uri = create_uri(LP, f'token{namespace_sep}{expr_index}', token_id - 1, namespace_sep)
                g.add((powla_token_uri, POWLA.prev, prev_token_uri))
                g.add((prev_token_uri, POWLA.next, powla_token_uri))
    
            # Dependency relation
            if token.head:
                token_head = int(token.head)
                if token_head != 0:                   
                    dep_relation_uri = create_uri(LP, f'deprel{namespace_sep}{expr_index}', f'{token_head}-{token_id}', namespace_sep)
                    powla_head_uri = create_uri(LP, f'token{namespace_sep}{expr_index}', token_head, namespace_sep)                                        
                    g.add((dep_relation_uri, RDF.type, POWLA.relation))
                    ud_deprel = token.deprel.split(':')[0]
                    deprel_uri = create_uri(LP, f'deprel', ud_deprel, namespace_sep)
                    g.add((deprel_uri, RDF.type, URIRef(f'https://universaldependencies.org/u/dep/{ud_deprel}')))
                    g.add((deprel_uri, RDFS.label, Literal(f'UD {ud_deprel}'))) 
                    g.add((dep_relation_uri, RDF.type, deprel_uri))                    
                    g.add((dep_relation_uri, RDFS.label, Literal(f'deprel: {token.deprel}')))
                    g.add((dep_relation_uri, POWLA.hasLayer, dep_layer))
                    g.add((dep_relation_uri, POWLA.hasSource, powla_head_uri))
                    g.add((dep_relation_uri, POWLA.hasTarget, powla_token_uri))
                    g.add((dep_relation_uri, POWLA.hasLabel, Literal(token.deprel)))    
            
            
            # UD annotations from conllu
            ud_node_uri = create_uri(LP, f'ud{namespace_sep}{expr_index}', token_id, namespace_sep)
            g.add((ud_node_uri, RDFS.label, Literal(f'UD annotation of {token_form}')))
            g.add((ud_node_uri, RDF.type, OA.annotation))
            g.add((ud_node_uri, POWLA.hasLayer, ud_layer))
            g.add((ud_node_uri, OA.hasTarget, powla_token_uri))
            
            # POS from conllu
            pos_node_uri = create_uri(LP, 'pos', f'{token.upos}', namespace_sep)
            g.add((pos_node_uri, RDFS.label, Literal(f'POS={token.upos}')))
            g.add((pos_node_uri, RDF.type, URIRef(f'https://universaldependencies.org/u/pos/{token.upos}')))
            g.add((URIRef(f'https://universaldependencies.org/u/pos/{token.upos}'), RDFS.label, Literal(f'UD {token.upos}')))
            g.add((ud_node_uri, OA.hasBody, pos_node_uri))
    
            # Feats from conllu
            feats = dict(token.feats)
            for feat in feats:
                for f in feats[feat]:
                    feat_node_uri= create_uri(LP, 'feat', f'{feat.capitalize()}_{f.capitalize()}', namespace_sep)
                    g.add((feat_node_uri, RDFS.label, Literal(f'{feat.capitalize()}={f.capitalize()}')))
                    g.add((feat_node_uri, RDF.type, URIRef(f'https://universaldependencies.org/u/feat/{feat.capitalize()}#{f.capitalize()}')))
                    g.add((URIRef(f'https://universaldependencies.org/la/feat/{feat.capitalize()}#{f.capitalize()}'), RDFS.label, Literal(f'{feat.capitalize()}={f.capitalize()}')))
                    g.add((ud_node_uri, OA.hasBody, feat_node_uri))
    
            if token.form == 'Plato': # ad hoc, can be generalized?
                plato_token_uri = powla_token_uri                
            
            # Link to LiLa lemma
            try :
                for sent in lila_linking[row['id']]['sentences']:
                    for lila_token in sent:
                        if lila_token['token'] == token_form:
                            if len(lila_token['linking']) > 0:
                                lila_link_info = lila_token['linking'][0].split(":")
                                lila_lemma_id = lila_link_info[1]                
                                lila_lemma_uri = URIRef(f'http://lila-erc.eu/data/id/lemma/{lila_lemma_id}')  
                                g.add((lila_lemma_uri, RDF.type, LILA.lemma))
                                g.add((lila_lemma_uri, RDFS.label, Literal(lila_token['lemma'])))
                                g.add((lila_lemma_uri, DC.title, Literal(lila_token['lemma'])))
                                g.add((powla_token_uri, LILA.hasLemma, lila_lemma_uri))
                            else:
                                print(latin_expr + " "+ " unmatched token in LiLa " + token.form + " " + lila_token['token'])
            except :
                print(latin_expr + " "+ " error linking " + token.form)

try:
    g.add((plato_token_uri, RDFS.seeAlso, plato_uri))
except:
    print("Plato error")

# Add links between concepts
for key, value in linked_concepts_dict.items():
     concept_uri = value['uri']
     for link in value['links']:
         g.add((concept_uri, SKOS.related, linked_concepts_dict[link]['uri']))

a dicto secundum quid ad dictum simpliciter
a dicto simpliciter ad dictum secundum quid
a fortiori
a necesse ad esse valet consequentia
a posteriori
a priori
ab esse ad posse valet consequentia
ab ovo
actus purus
ad hoc
ad hoc hypothesis
ad infinitum
amicus Plato sed magis amica veritas
amor fati
anima mundi
argumentum ad baculum
argumentum ad hominem
argumentum ad populum
ars gratia artis
barbara
bellum omnium contra omnes
causa sine qua non
causa sui
ceteris paribus
characteristica universalis
cogito, ergo sum
conditio sine qua non
consequentia mirabilis
creatio ex nihilo
credo quia absurdum est
credo quia impossibile est
credo ut intelligam
cui bono?
de dicto
de facto
de gustibus non disputandum
de jure
de re
deus sive natura
dictum de omni et nullo
do ut des
dubito, ergo cogito, ergo sum
dum spiro, spero
ens causa sui
ens rationis
ens realissimum
entia non sunt multiplicanda praeter necessitatem
esse est percipi
ex falso quodlibet
ex nihilo nihil fit
ex post facto
ex vi terminorum


In [ ]:
#print(g.serialize(format="turtle"))

### Serialize triplestore

In [109]:
g.serialize(format="turtle", destination="Latin-Philosophical-Expression.ttl")

<Graph identifier=N1d9384cb73b24aaf8cb70c524b9ab0ae (<class 'rdflib.graph.Graph'>)>